In [ ]:
def _set_true_buy_qty(self, df):
        refid_counts =  df.refid.value_counts()
        refid_repeats = refid_counts[refid_counts>1].index
        for j in refid_repeats:
            temp_df = df[df["refid"]==j]
            if temp_df["status"].to_list() != ["Sold","Sold"]:
                mask = (df["status"]=="Bought") & (df["refid"] == j)
                df["qty"].loc[mask] = temp_df["qty"].sum()
        return df

In [2]:
import datetime as dt
import numpy as np

In [3]:
from transactions import get_event_transactions, remove_unfilled_txn_qty
from transactions_cleaner import remove_cancelled_orders

In [34]:
df1 = get_event_transactions(12401)
# df1["createdat"] = df1["createdat"].dt.time

In [35]:
df1.head()

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
1,2022-11-21,2022-11-21 21:01:38.452000+05:30,12401,od_18env14I,312.0,Cancelled 4.00 shares x N (Rs. 78.00 each),Cancelled,4.0,N,78.0
8,2022-11-21,2022-11-21 20:55:06.808000+05:30,12401,od_H50suOZW,80.0,Sold 1.00 shares x N (Rs. 80.00 each),Sold,1.0,N,80.0
9,2022-11-21,2022-11-21 20:54:58.720000+05:30,12401,od_H50suOZW,80.0,Sold 1.00 shares x N (Rs. 80.00 each),Sold,1.0,N,80.0
31,2022-11-21,2022-11-21 20:28:56.305000+05:30,12401,od_H50suOZW,80.0,Sold 1.00 shares x N (Rs. 80.00 each),Sold,1.0,N,80.0
53,2022-11-21,2022-11-21 20:11:04.490000+05:30,12401,od_18env14I,-312.0,Bought 4.00 shares x N (Rs. 78.00 each),Bought,4.0,N,78.0


In [12]:
df1[df1["refid"]=="od_ZCMmisHm"]

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
136,2022-11-17,2022-11-17 21:15:59.873000+05:30,12179,od_ZCMmisHm,24.75,Sold 1.00 shares x Y (Rs. 25.00 each),Sold,1.0,Y,25.0
137,2022-11-17,2022-11-17 21:15:59.873000+05:30,12179,od_ZCMmisHm,24.75,Sold 1.00 shares x Y (Rs. 25.00 each),Sold,1.0,Y,25.0


In [5]:
def remove_cancelled_orders(df):
    #only un-comment prints to figure waht is happenin
    og_df = df.copy()
    del_row_indices = []
    for ref in og_df["refid"].unique():
        temp = og_df[og_df["refid"]==ref]
        if (temp.shape[0]>1) and (len(temp["status"].unique())>1) and (temp["status"].unique()[0] != "Sold"):
            # print(temp[["createdat","refid","amount","status","qty","asset","price"]])
            if (len(temp["status"].unique())==2) and (temp["status"].unique()[0] == "Cancelled") and (temp["status"].unique()[1] == "Bought"):
                cancel_index = temp[temp["status"]=="Cancelled"].index[0]
                bought_index = temp[temp["status"]=="Bought"].index[0]
                if (temp["qty"].loc[cancel_index] == temp["qty"].loc[bought_index]) and (temp["price"].loc[cancel_index] == temp["price"].loc[bought_index]):
                    del_row_indices.append(cancel_index)
                    del_row_indices.append(bought_index)
                    # print(del_row_indices)
                elif (temp["qty"].loc[cancel_index] != temp["qty"].loc[bought_index]):
                    new_amount = temp["amount"].loc[cancel_index] + temp["amount"].loc[bought_index]
                    new_qty = temp["qty"].loc[bought_index] - temp["qty"].loc[cancel_index]

                    og_df["qty"].loc[bought_index] = new_qty
                    og_df["amount"].loc[bought_index] = new_amount

                    # print(og_df.loc[bought_index])
                    del_row_indices.append(cancel_index)
                    # print(del_row_indices)

            else:
                print("ErrorErrorErrorErrorErrorErrorErrorErrorErrorError")
                raise Exception
    og_df = og_df.drop(del_row_indices)
    if np.round(df["amount"].sum(),2) == np.round(df["amount"].sum(),2):
        print("Amount sum matches")
    else:
        print("Amount sum un-matched")
        raise Exception
    return og_df

In [31]:
df2 = remove_cancelled_orders(df1)

Amount sum matches


In [56]:
df1["amount"].sum()

-1206.6600000000003

In [55]:
df2["amount"].sum()

-1206.66

In [53]:
for row in df1.index:
    print(row)

5
6
7
8
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
70
71
72
73
74
75
76
77
78
79
81
83
84
85
86
88
89
92
93
98
99
102
103
107
108
109
111
112
113
116
118
119
127
128
130
131
133
137
138
141
142
144
145
147
153
154
156
163
164
169
170
177
184
185
186
187
188
190
191
195
197
199
200
201
202
205
206
209
210
219
220
221
222
223
224
225
226
234
235
237
238
246
251
254
255
259
260
263
264
265
266
269
270
287
288
296
297
302
303
307
308
309
316
317
324
325
327
328
332
333
338
339
343
344
345
346
347
348
352
353
355
359
362
363
367
368
369
372
390
391
394
395
400
405
406
409
418
441
442
445
446
454
455
469
470
478
479
480
481
493
494
495
496
497
498
502
503
507
508
512
513
516
517
518
519
520
521
537
538
546
547
557
558
559
560
563
564
567
568
575
576
582
583
590
595
596
604
605
606
607
611
612
616
617
621
622
630
631
636
637
643


In [96]:
df2 = df2.iloc[::-1]

In [66]:
df2

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
710,2022-11-17,2022-11-17 15:22:52.747000+05:30,12177,od_Ti03zZUK,-80.00,Bought 1.00 shares x Y (Rs. 80.00 each),Bought,1.0,Y,80.0
691,2022-11-17,2022-11-17 15:30:06.807000+05:30,12177,od_vS6eUuoO,-77.77,Bought 1.00 shares x Y (Rs. 77.00 each),Bought,1.0,Y,77.0
684,2022-11-17,2022-11-17 15:34:40.780000+05:30,12177,od_h7qObPI0,158.40,Sold 2.00 shares x Y (Rs. 80.00 each),Sold,2.0,Y,80.0
650,2022-11-17,2022-11-17 15:48:27.713000+05:30,12177,od_8KidsuJI,-85.00,Bought 2.00 shares x Y (Rs. 85.00 each),Bought,1.0,Y,85.0
642,2022-11-17,2022-11-17 16:02:54.555000+05:30,12177,od_4QbmlARo,82.00,Sold 1.00 shares x Y (Rs. 82.00 each),Sold,1.0,Y,82.0
...,...,...,...,...,...,...,...,...,...,...
181,2022-11-17,2022-11-17 21:29:48.690000+05:30,12177,od_wARzWf2C,87.12,Sold 1.00 shares x Y (Rs. 88.00 each),Sold,1.0,Y,88.0
179,2022-11-17,2022-11-17 21:31:50.583000+05:30,12177,od_4ITlr9lY,87.12,Sold 1.00 shares x Y (Rs. 88.00 each),Sold,1.0,Y,88.0
170,2022-11-17,2022-11-17 21:35:23.419000+05:30,12177,od_OPUrjmGQ,88.00,Sold 1.00 shares x Y (Rs. 88.00 each),Sold,1.0,Y,88.0
168,2022-11-17,2022-11-17 21:36:17.234000+05:30,12177,od_MFkIgvhD,-352.00,Bought 4.00 shares x Y (Rs. 88.00 each),Bought,4.0,Y,88.0


In [67]:
df2.to_csv("fairpricedata/tradelogs_12177.csv")

In [7]:
df2

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
2,2022-11-21,2022-11-21 19:14:20.385000+05:30,12401,od_UmOBI4kJ,-315.00,Bought 5.00 shares x N (Rs. 63.00 each),Bought,5.0,N,63.0
8,2022-11-21,2022-11-21 19:08:36.770000+05:30,12401,od_kNzsiKe4,37.00,Sold 1.00 shares x Y (Rs. 37.00 each),Sold,1.0,Y,37.0
19,2022-11-21,2022-11-21 19:02:36.640000+05:30,12401,od_kNzsiKe4,37.00,Sold 1.00 shares x Y (Rs. 37.00 each),Sold,1.0,Y,37.0
39,2022-11-21,2022-11-21 18:51:30.840000+05:30,12401,od_cCHOGHlv,28.00,Sold 1.00 shares x Y (Rs. 28.00 each),Sold,1.0,Y,28.0
49,2022-11-21,2022-11-21 18:42:55.791000+05:30,12401,od_1f5UXIaU,-27.00,Bought 5.00 shares x Y (Rs. 27.00 each),Bought,1.0,Y,27.0
51,2022-11-21,2022-11-21 18:42:50.074000+05:30,12401,od_r267Mlq4,-54.54,Bought 5.00 shares x Y (Rs. 27.00 each),Bought,2.0,Y,27.0
87,2022-11-21,2022-11-21 18:13:52.653000+05:30,12401,od_YKuHVokJ,64.00,Sold 1.00 shares x N (Rs. 64.00 each),Sold,1.0,N,64.0
91,2022-11-21,2022-11-21 18:12:01.630000+05:30,12401,od_YKuHVokJ,64.00,Sold 1.00 shares x N (Rs. 64.00 each),Sold,1.0,N,64.0
92,2022-11-21,2022-11-21 18:11:36.860000+05:30,12401,od_YKuHVokJ,64.00,Sold 1.00 shares x N (Rs. 64.00 each),Sold,1.0,N,64.0
93,2022-11-21,2022-11-21 18:11:20.408000+05:30,12401,od_YKuHVokJ,64.00,Sold 1.00 shares x N (Rs. 64.00 each),Sold,1.0,N,64.0


In [99]:
mask_buy = df2["status"]== "Bought"
mask_yes = df2["asset"]== "Y"
df2_yb = df2[mask_buy & mask_yes]

In [100]:
df2_yb

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
262,2022-11-18,2022-11-18 16:53:12.172000+05:30,12230,od_GlVK1PdW,-360.0,Bought 4.00 shares x Y (Rs. 90.00 each),Bought,4.0,Y,90.0
181,2022-11-18,2022-11-18 17:52:30.446000+05:30,12230,od_3PJFMC1u,-86.0,Bought 4.00 shares x Y (Rs. 86.00 each),Bought,1.0,Y,86.0
164,2022-11-18,2022-11-18 18:04:46.107000+05:30,12230,od_MXwlnD4T,-88.0,Bought 3.00 shares x Y (Rs. 88.00 each),Bought,1.0,Y,88.0
146,2022-11-18,2022-11-18 18:14:28.796000+05:30,12230,od_N5owi1Y5,-270.0,Bought 3.00 shares x Y (Rs. 90.00 each),Bought,3.0,Y,90.0
43,2022-11-18,2022-11-18 19:12:34.038000+05:30,12230,od_vbvUlJVd,-180.0,Bought 2.00 shares x Y (Rs. 90.00 each),Bought,2.0,Y,90.0


In [101]:
(df2_yb["price"]*df2_yb["qty"]).sum() / df2_yb["qty"].sum()

89.45454545454545

In [7]:
df3 = get_event_transactions(12230)
og_df = df3.copy()

In [8]:
for ref in og_df["refid"].unique():
    temp = og_df[og_df["refid"]==ref]
    if (temp.shape[0]>1) and (len(temp["status"].unique())>1) and (temp["status"].unique()[0] != "Sold"):
        print(temp)

           date                        createdat  event_id        refid  \
929  2022-11-18 2022-11-18 20:52:25.877000+05:30     12230  od_pBYM7Wkx   
955  2022-11-18 2022-11-18 20:41:27.316000+05:30     12230  od_pBYM7Wkx   

     amount                                 order_detail     status  qty  \
929   324.0   Cancelled 4.00 shares x Y (Rs. 81.00 each)  Cancelled  4.0   
955  -324.0      Bought 4.00 shares x Y (Rs. 81.00 each)     Bought  4.0   

    asset  price  
929     Y   81.0  
955     Y   81.0  
            date                        createdat  event_id        refid  \
960   2022-11-18 2022-11-18 20:37:11.697000+05:30     12230  od_vbvUlJVd   
1159  2022-11-18 2022-11-18 19:12:34.038000+05:30     12230  od_vbvUlJVd   

      amount                                 order_detail     status  qty  \
960    180.0   Cancelled 2.00 shares x Y (Rs. 90.00 each)  Cancelled  2.0   
1159  -180.0      Bought 2.00 shares x Y (Rs. 90.00 each)     Bought  2.0   

     asset  price  
960    

In [10]:
df2

NameError: name 'df2' is not defined

In [25]:
df2["refid"].value_counts()

od_dghQbMHa    4
od_l6SCev9t    4
od_vVVphl0m    3
od_Jiw07AVG    3
od_7Yj6vPy1    3
              ..
od_Zy7tynBd    1
od_wo8mGjX3    1
od_pVMoJQuu    1
od_gy3knuiN    1
od_D4TNkWOt    1
Name: refid, Length: 68, dtype: int64

In [32]:
z = df2[df2["refid"]=="od_TOEqNi0m"]

In [31]:
df2.index.values

array([  1,  48,  51,  52,  53,  54,  56,  58,  64,  79,  80,  95, 129,
       140, 144, 151, 158, 160, 166, 167, 168, 170, 171, 172, 174, 175,
       177, 178, 179, 190, 191, 192, 195, 196, 198, 204, 207, 213, 227,
       257, 308, 310, 315, 316, 317, 318, 322, 325, 326, 336, 337, 358,
       362, 363, 366, 374, 388, 389, 390, 391, 392, 394, 407, 426, 432,
       433, 447, 449, 452, 454, 457, 474, 477, 482, 486, 492, 504, 512,
       513, 514, 515, 516, 517, 526, 544, 580], dtype=int64)

In [11]:
z.shape

(1, 10)

In [33]:
z

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
7,2022-11-21,2022-11-21 20:06:06.483000+05:30,12400,od_TOEqNi0m,54.0,Sold 1.00 shares x Y (Rs. 54.00 each),Sold,1.0,Y,54.0


In [14]:
z.loc[2]["qty"]

5.0

In [18]:
z.loc[2,["qty"]] = 66

In [22]:
(z.loc[2]["qty"]).value()

AttributeError: 'numpy.float64' object has no attribute 'value'

In [17]:
z

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
2,2022-11-21,2022-11-21 19:14:20.385000+05:30,12401,od_UmOBI4kJ,-315.0,Bought 5.00 shares x N (Rs. 63.00 each),Bought,5.0,N,63.0


In [29]:
(z.loc[2,["qty"]]).values[0]

66.0

In [37]:
df1[df1["refid"]=="od_H50suOZW"]

,date,createdat,event_id,refid,amount,order_detail,status,qty,asset,price
8,2022-11-21,2022-11-21 20:55:06.808000+05:30,12401,od_H50suOZW,80.0,Sold 1.00 shares x N (Rs. 80.00 each),Sold,1.0,N,80.0
9,2022-11-21,2022-11-21 20:54:58.720000+05:30,12401,od_H50suOZW,80.0,Sold 1.00 shares x N (Rs. 80.00 each),Sold,1.0,N,80.0
31,2022-11-21,2022-11-21 20:28:56.305000+05:30,12401,od_H50suOZW,80.0,Sold 1.00 shares x N (Rs. 80.00 each),Sold,1.0,N,80.0
